In [ ]:
import numpy as np
import os
import sys

from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Concatenate, Embedding, Convolution1D,Dropout, Conv2D, Conv1D, Bidirectional, Permute
from keras.layers.wrappers import TimeDistributed

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.layers import add, dot, concatenate
import argparse

import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec
from keras.callbacks import EarlyStopping,TensorBoard, ModelCheckpoint
from keras.layers.recurrent import Recurrent

from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHeadAttention

import pickle as plk

from keras_bert import Tokenizer, extract_embeddings, load_trained_model_from_checkpoint
from keras_bert import AdamWarmup, calc_train_steps


gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.tensorflow_backend.set_session(sess)

In [ ]:
Dropout_rate = 0.35
lstm_cells = 256
classes = 6
batch = 32
epochs = 500
LEARNING_RATE = 1e-4

max_sentence_length = 139
bert_len = 768
len_vocab = 32000

In [ ]:
data_txt_tra = plk.load(open('../data_clean/split/data_txt_tra', 'rb'))
data_txt_pre_tra = plk.load(open('../data_clean/split/data_txt_pre_tra', 'rb'))

data_txt_tes = plk.load(open('../data_clean/split/data_txt_tes', 'rb'))
data_txt_pre_tes = plk.load(open('../data_clean/split/data_txt_pre_tes', 'rb'))

data_emos_tra = plk.load(open('../data_clean/split/data_emos_tra_', 'rb'))
data_emos_tes = plk.load(open('../data_clean/split/data_emos_tes', 'rb'))

In [ ]:
data = []
# noise = np.random.normal(0, 0.1, (384, 256))
loop_n = {0:3,1:2,2:1,3:2,4:2,5:1}
for (utt, utt_pre, emo) in zip(data_txt_tra, data_txt_pre_tra, data_emos_tra):
    for _ in range(loop_n[emo]):
        data.append([utt, utt_pre, emo])
        
np.random.shuffle(data)
data_txt_tra, data_txt_pre_tra, data_emos_tra = [], [], []
for (utt, utt_pre, emo) in data:
    data_txt_tra.append(utt)
    data_txt_pre_tra.append(utt_pre)
    data_emos_tra.append(emo)

In [ ]:
data_txt_tra = np.asarray(data_txt_tra)
data_txt_pre_tra = np.asarray(data_txt_pre_tra)

data_emos_tra = to_categorical(data_emos_tra, 6)
data_emos_tes = to_categorical(data_emos_tes, 6)

print('data_txt_tra:', np.shape(data_txt_tra))
print('data_txt_tes:', np.shape(data_txt_tes))

print('data_txt_pre_tra:', np.shape(data_txt_pre_tra))
print('data_txt_pre_tes:', np.shape(data_txt_pre_tes))

print('data_emos_tra:', np.shape(data_emos_tra))
print('data_emos_tes:', np.shape(data_emos_tes))

In [ ]:
def SeMemNN_ser(input_shape, class_count):
    decay_steps, warmup_steps = calc_train_steps(
        input_shape[0],
        batch_size=batch,
        epochs=epochs,
        warmup_proportion=0.01
    )
    
    pre_txt = Input((max_sentence_length,))
    txt = Input((max_sentence_length,))
    
    input_encoder_m = Sequential()
    input_encoder_m.add(Embedding(input_dim=len_vocab,
                                  output_dim=256))
#     input_encoder_m.add(LSTM(256, return_sequences=True))
    input_encoder_m.add(Dropout(Dropout_rate))
    # output: (samples, max_len_tra_des_sent, embedding_dim)

    input_encoder_c = Sequential()
    input_encoder_c.add(Embedding(input_dim=len_vocab,
                                  output_dim=max_sentence_length))
#     input_encoder_c.add(LSTM(max_sentence_length, return_sequences=True))
    input_encoder_c.add(Dropout(Dropout_rate))
    # output: (samples, max_len_tra_des_sent, max_len_tra_title_sent)

    # embed the title into a sequence of vectors
    pre_encoder = Sequential()
    pre_encoder.add(Embedding(input_dim=len_vocab,
                                   output_dim=256,
                                   input_length=max_sentence_length))
#     pre_encoder.add(LSTM(256, return_sequences=True))
    pre_encoder.add(Dropout(Dropout_rate))
    # output: (samples, max_len_tra_title_sent, embedding_dim)


    input_encoded_m = input_encoder_m(txt)
    input_encoded_c = input_encoder_c(pre_txt)
    pre_encoded = pre_encoder(pre_txt)

    # shape: (samples, max_len_tra_des_sent, max_len_tra_title_sent)
    match = dot([input_encoded_m, pre_encoded], axes=(2, 2))
    match = Activation('softmax')(match)

    response = add([match, input_encoded_c])  # (samples, max_len_tra_des_sent, max_len_tra_title_sent)
    response = Permute((2, 1))(response)  # (samples, max_len_tra_title_sent, max_len_tra_des_sent)

    R = concatenate([response, pre_encoded])
    # R = Bidirectional(LSTM(256))(R)
    R = Bidirectional(LSTM(256,return_sequences=True))(R)
    R = MultiHeadAttention(head_num=8)(R)
    R = Bidirectional(LSTM(256))(R)

    R = Dropout(0.35)(R)
    R = Dense(classes)(R)  # (samples, vocab_size)
    result = Activation('softmax')(R)


    model = Model([pre_txt, txt], result)
    model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['mae', 'mse', 'acc'])

    return model

In [ ]:
model = SeMemNN_ser(data_txt_tra.shape, 6)
model.summary()

In [ ]:
file_path_root = './text/'
model_file = file_path_root+'text_model.h5'
callback_list = [
                    TensorBoard(log_dir=file_path_root),
                    EarlyStopping(
                        monitor='val_acc',
                        patience=150,
                        verbose=1,
                        mode='auto'
                    ),
                    ModelCheckpoint(
                        filepath=model_file,
                        monitor='val_acc',
                        save_best_only='True',
                        verbose=1,
                        mode='auto',
                        period=1
                    )
                    ]

In [ ]:
training = model.fit([data_txt_tra, data_txt_pre_tra], 
          data_emos_tra,
          batch_size=batch,
          epochs=epochs,
          callbacks=callback_list,      
          validation_data=([data_txt_tes, data_txt_pre_tes], 
                                 data_emos_tes))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd


model = load_model(model_file, custom_objects = {'SeqSelfAttention': SeqSelfAttention, 'MultiHeadAttention':MultiHeadAttention})

predicted_test_labels = model.predict([data_txt_tes, data_txt_pre_tes]).argmax(axis=1)
numeric_test_labels = np.array(data_emos_tes).argmax(axis=1)

report_filename = file_path_root+'result_4digits.txt'

with open(report_filename, 'w', encoding='utf-8') as f:
    print(classification_report(numeric_test_labels, predicted_test_labels, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru'], digits=4), file=f)
    print(classification_report(numeric_test_labels, predicted_test_labels, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru'], digits=4))
    
labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
print('   '+' '.join(labels))
cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)



nor_cm = []

for i in range(6):
    row_sum = cm[i].sum()
#     print(row_sum)
    l_n = []
    for j in range(6):
        l_n.append(cm[i][j]/row_sum)
    nor_cm.append(l_n)
    
df_cm = pd.DataFrame(nor_cm, index = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']],
                  columns = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']])

sn.heatmap(df_cm,  annot=True)
plt.savefig(file_path_root+'cm.jpg')

cm = np.transpose(cm)

nor_cm = []
for i in range(6):
    row_sum = cm[i].sum()
#     print(row_sum)
    l_n = []
    for j in range(6):
        l_n.append(cm[i][j]/row_sum)
    nor_cm.append(l_n)
    
df_cm = pd.DataFrame(nor_cm, index = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']],
                  columns = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']])

sn.heatmap(df_cm,  annot=True)
plt.savefig(file_path_root+'cm_precision.jpg')